In [27]:

import os
#change to your service principle (Workspace identity) ClientID, tenantID and clientsecret
#customer should setup Azure Key Vault and store secret in AKV instead of hardcoding here
#https://www.datasarva.com/fabric-notebook-azurekeyvault/
AKV_ENDPOINT = "https://xxx.vault.azure.net/"
os.environ["AZURE_CLIENT_ID"] = mssparkutils.credentials.getSecret(AKV_ENDPOINT, 'AZURE-CLIENT-ID')
os.environ["AZURE_TENANT_ID"] = mssparkutils.credentials.getSecret(AKV_ENDPOINT, 'AZURE-TENANT-ID')
os.environ["AZURE_CLIENT_SECRET"] = mssparkutils.credentials.getSecret(AKV_ENDPOINT, 'AZURE-CLIENT-SECRET')
#os.environ["AZURE_CLIENT_ID"] = ""
#os.environ["AZURE_TENANT_ID"] = ""
#os.environ["AZURE_CLIENT_SECRET"] = ""

#change to your workspace name
workspaceidty = "IT-team-workspace"

#cut off date the data ingestion CIJ data, any record before this date will be ignored
cutoffdate = "2024-05-09 09:46:55"

StatementMeta(, 800760f7-f1b5-4947-b676-c7db169eab19, 29, Finished, Available, Finished)

In [28]:
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType, StructField, StringType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("PostgreSQL Example") \
    .config("spark.jars", "/path/to/postgresql-42.2.18.jar") \
    .getOrCreate()

# Define the JDBC URL and connection properties
jdbc_url = "jdbc:postgresql://xxx.postgres.database.azure.com:5432/fabricoutput"

from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

token = credential.get_token("https://ossrdbms-aad.database.windows.net/.default").token

connection_properties = {
    "user": workspaceidty,
    "password": token,
    "driver": "org.postgresql.Driver"
}

#print("token = ", token)

StatementMeta(, 800760f7-f1b5-4947-b676-c7db169eab19, 30, Finished, Available, Finished)

In [53]:
# just for debug
schema = StructType([
    StructField("lasttimestamp", StringType(), True)
])

# Create the DataFrame
data = [(cutoffdate,)]
write_ts_df = spark.createDataFrame(data, schema)

# Write DataFrame to PostgreSQL
write_ts_df.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "public.last_ts_clicked") \
    .options(**connection_properties) \
    .mode("overwrite") \
    .save()

StatementMeta(, 5942b3cd-cfad-477e-aaca-bb96a1f003e1, 55, Finished, Available, Finished)

In [29]:
# Load the Parquet files from the folder
df = spark.read.parquet("Files/EmailClicked")

# Show the DataFrame
#df.show()

StatementMeta(, 800760f7-f1b5-4947-b676-c7db169eab19, 31, Finished, Available, Finished)

In [30]:
# Show the schema of the DataFrame
# df.printSchema()

StatementMeta(, 800760f7-f1b5-4947-b676-c7db169eab19, 32, Finished, Available, Finished)

In [31]:

# Query to check if the table exists
query = """
SELECT EXISTS (
    SELECT 1
    FROM information_schema.tables 
    WHERE table_schema = 'public'
    AND table_name = 'last_ts_clicked'
) AS table_exists
"""

# Execute the query
dfchecktbl = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("query", query) \
    .options(**connection_properties) \
    .load()

# Check if the table exists
table_exists = dfchecktbl.collect()[0]["table_exists"]

if not table_exists:
    print("table_exists = NO", )
    # Define the schema
    schema = StructType([
        StructField("lasttimestamp", StringType(), True)
    ])

    # Create the DataFrame
    data = [(cutoffdate,)]
    write_ts_df = spark.createDataFrame(data, schema)

    # Write DataFrame to PostgreSQL
    write_ts_df.write \
      .format("jdbc") \
      .option("url", jdbc_url) \
      .option("dbtable", "public.last_ts_clicked") \
      .options(**connection_properties) \
      .mode("overwrite") \
      .save()
    
    last_ts = cutoffdate
    print("last_ts = ", last_ts)      
else:
    print("table_exists = YES", )
    # Read the table
    load_ts_df = spark.read \
        .format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", "public.last_ts_clicked") \
        .options(**connection_properties) \
        .load()

    # Show the first row value
    first_row = load_ts_df.first()
    last_ts = first_row["lasttimestamp"]
    print("last_ts = ", last_ts)
    load_ts_df.show()

StatementMeta(, 800760f7-f1b5-4947-b676-c7db169eab19, 33, Finished, Available, Finished)

table_exists = NO
last_ts =  2024-05-09 09:46:55


In [32]:
# Get a list of all column names
#columns = df.columns
#print("Columns: ", columns)

# Rename columns
#df = df.withColumnRenamed("old_name1", "new_name1")    

# Select only the "ActivityId" and "Timestamp" columns
#selected_df = df.select("ActivityId", "Timestamp")

# Show the DataFrame with only the selected columns
#selected_df.show()

StatementMeta(, 800760f7-f1b5-4947-b676-c7db169eab19, 34, Finished, Available, Finished)

In [33]:
from pyspark.sql.functions import col


# Sort the DataFrame by Timestamp in ascending order
sorted_df = df.orderBy(col("Timestamp").asc())

# Filter the DataFrame to include only records with Timestamp > '2024-05-09 09:46:55'
dupfiltered_df = sorted_df.filter(col("Timestamp") > last_ts)

# Deduplicate the DataFrame
print("before dedup count = ", dupfiltered_df.count())
filtered_df = dupfiltered_df.dropDuplicates()
print("after dedup count = ", filtered_df.count())

# Get the last record's Timestamp
newlast_ts = filtered_df.collect()[-1]['Timestamp']

if not isinstance(newlast_ts, str):
    newlast_ts = str(newlast_ts)

print("newlast_ts", newlast_ts)

# Show the filtered DataFrame
#filtered_df.show()

StatementMeta(, 800760f7-f1b5-4947-b676-c7db169eab19, 35, Finished, Available, Finished)

before dedup count =  5
after dedup count =  5
newlast_ts 2024-05-09 09:47:15


In [34]:
# Count the number of records in the filtered DataFrame
filtered_record_count = filtered_df.count()

# Print the count
print("Number of records:", filtered_record_count)

StatementMeta(, 800760f7-f1b5-4947-b676-c7db169eab19, 36, Finished, Available, Finished)

Number of records: 5


In [35]:
# Count the number of records in the filtered DataFrame
record_count = df.count()

# Print the count
print("Number of records:", record_count)

StatementMeta(, 800760f7-f1b5-4947-b676-c7db169eab19, 37, Finished, Available, Finished)

Number of records: 6


In [36]:
# Write DataFrame to PostgreSQL
filtered_df.write \
  .format("jdbc") \
  .option("url", jdbc_url) \
  .option("dbtable", "public.emailclicked") \
  .options(**connection_properties) \
  .mode("append") \
  .save()


StatementMeta(, 800760f7-f1b5-4947-b676-c7db169eab19, 38, Finished, Available, Finished)

In [37]:
# just for debug
# Validate the records in db
# Read the table back from PostgreSQL
read_updated_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "public.emailclicked") \
    .options(**connection_properties) \
    .load()

# Get the count of records in the original DataFrame
original_count = filtered_df.count()

# Get the count of records in the table read from PostgreSQL
added_count = read_updated_df.count()

# Validate the counts
if original_count == added_count:
    print("Validation successful: The number of records matches.")
else:
    print(f"Validation failed: Original count = {original_count}, added_count = {added_count}")

StatementMeta(, 800760f7-f1b5-4947-b676-c7db169eab19, 39, Finished, Available, Finished)

Validation successful: The number of records matches.


In [38]:
# Write new timestamp 

schema = StructType([
    StructField("lasttimestamp", StringType(), True)
])
print("newlast_ts", newlast_ts)
data = [(newlast_ts,)]
newts_df = spark.createDataFrame(data, schema)


newts_df.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "public.last_ts_clicked") \
    .options(**connection_properties) \
    .mode("overwrite") \
    .save()



StatementMeta(, 800760f7-f1b5-4947-b676-c7db169eab19, 40, Finished, Available, Finished)

newlast_ts 2024-05-09 09:47:15


In [39]:
# just for debug
# Validate the timestamp records in db
read_ts_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "last_ts_clicked") \
    .options(**connection_properties) \
    .load()


# Show the first row value
first_row = read_ts_df.first()
newlast_ts = first_row["lasttimestamp"]
print("newlast_ts = ", newlast_ts)


StatementMeta(, 800760f7-f1b5-4947-b676-c7db169eab19, 41, Finished, Available, Finished)

newlast_ts =  2024-05-09 09:47:15
